In [4]:
import pandas as pd
import numpy as np
import sklearn
import datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq

from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
from surprise import Reader, Dataset
from surprise import KNNBaseline
from surprise.model_selection import GridSearchCV

from surprise import SVD, BaselineOnly, CoClustering

from tqdm import tqdm
import pickle
from surprise import dump

In [6]:
train = pq.read_table('train_without_None.gzip').to_pandas()

test_private = pq.read_table('test_private_users_mfti.parquet ').to_pandas()

In [10]:
#Load the model from disk
predictions, algo = dump.load('./dump_file')

In [7]:
def generate_testset(user_id_, vacancy_ids_to_pred):
    return [(user_id_, vacancy_id, 0) for vacancy_id in vacancy_ids_to_pred]

In [26]:
def prepare_date(train, test_private, model):
    
    # создаем словарь cookie:user и создаем новый столбец в тесте соответсвующий user_id
    
    dict_cookie_to_user={i:j for i,j in zip(train.cookie_id, train.user_id_filled)}
    test_private['user_id']=test_private.cookie_id.apply(lambda x: dict_cookie_to_user[x])

    
    #собираем информацией о всех вакансиях которые смотрел user
    
    # по информации с вакансиями по которым были сделаны действия пользователя,
    # имеющими в истории действия по вакансиям, которые смотрел наш пользователь
    
    n_items=5
    new_column=[]
    
    for i in tqdm(range(len(test_private))):
           
        user_saw=set(train.loc[train["user_id_filled"] == test_private.user_id[i], "vacancy_id_"])
        all_same_user=list(train.user_id_filled[train.vacancy_id_.isin(user_saw)])
        all_vacansy_saw_same_user=list(train.vacancy_id_[train.user_id_filled.isin(all_same_user)])
        
        
        test_set=generate_testset(test_private['user_id'][i], (set(all_vacansy_saw_same_user)-user_saw))
        #предсказание
        predictions = model.test(test_set)

        #топ n лучшие вакансии на основе предсказанных рейтингов
        pred_ratings = np.array([pred.est for pred in predictions])   
        index_max = (-pred_ratings).argsort()[:n_items]
        
        new_column.append(np.array(list(set(all_vacansy_saw_same_user)-user_saw))[index_max])
    
    test_private['testset']=new_column

    
    
    return test_private

In [16]:
def generate_recommendation(model, user_id_, vacancy_ids_to_pred, n_items):

    test_set = [(user_id_, vacancy_id, 0) for vacancy_id in vacancy_ids_to_pred]

    #предсказание
    predictions = model.test(test_set)

    #топ n лучшие вакансии на основе предсказанных рейтингов
    pred_ratings = np.array([pred.est for pred in predictions])   
    index_max = (-pred_ratings).argsort()[:n_items]

    return vacancy_ids_to_pred[index_max]

In [27]:
test_after_prepare=prepare_date(train, test_private, algo)

100%|██████████| 3086/3086 [1:59:32<00:00,  2.32s/it]  


In [30]:
df=pd.DataFrame(list(zip( test_after_prepare.cookie_id, test_after_prepare.testset)), columns = ['cookie_id', 'vacancy_id_'])
df

,cookie_id,vacancy_id_
0,0018914ba3e54011b28fa715583d3354,"[208285, 233668, 246521, 180382, 174953]"
1,0035c298d8c64f368ae730a9cca9bb20,"[180382, 174953, 221920, 224323, 193372]"
2,00956458877448ec9fba87fb97443fdf,"[184067, 122785, 164086, 233668, 182044]"
3,0099387c921b41e7bae6c99dd8254b60,"[239245, 233668, 246521, 253555, 180382]"
4,009f65e8ae99413a8da94a491320580a,"[103320, 180382, 157311, 174953, 221920]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[239245, 184067, 164591, 230397, 178460]"
3082,ffbc08b528c64f22996873fc63872202,"[184067, 200208, 182044, 246521, 180382]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[122785, 220169, 200200, 181459, 180473]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[239245, 184067, 164591, 122785, 200208]"


In [32]:
df.to_parquet(f'test_private_users_mfti_predict_surpice.parquet')